In [1]:
from tools25 import *

In [2]:
from concurrent.futures import ThreadPoolExecutor

In [3]:
import logging
logging.basicConfig(filename='log/class_3_run.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
s = login()

b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'


In [5]:
# 获取指定数据集指定类型字段 的id 
fundamental6 = get_datafields(s, dataset_id='fundamental6')
fundamental6_filter = fundamental6[fundamental6['type']=='MATRIX']
datafields_list_fundamental6 = fundamental6_filter['id'].values 
# datafields_list_fundamental6

In [6]:
# 确定 alpha 模板 提取并定义其中可以替换的部分 
# alpha 模板 group_rank(ts_rank({fundamental model data}, 252), industry)  其中可以替换的部分包括 group_rank ts_rank {fundamental model data} 252 industry 
group_compare_op = ['group_rank', 'group_zscore', 'group_neutralize'] # 其他 group_ops 
ts_compare_op = ['ts_rank', 'ts_zscore', 'ts_av_diff', 'ts_sum', 'ts_scale', 'ts_arg_max'] # 其他 ts_ops： ts_min 
company_fundamental = datafields_list_fundamental6
days = [252, 504]
group = ['market', 'industry', 'subindustry', 'sector'] # 其他group可选值，包括 densify(fields) 

alpha_expressions = []
for group_op in group_compare_op:
    for ts_op in ts_compare_op:
        for datafield in company_fundamental:
            for day in days:
                for g in group:
                    alpha_expression = f'{group_op}({ts_op}({datafield}, {day}), {g})'
                    alpha_expressions.append(alpha_expression)

print(f'alpha_expressions length: {len(alpha_expressions)}') 
alpha_expressions[: 10] # 打印前10个表达式

alpha_expressions length: 82656


['group_rank(ts_rank(assets, 252), market)',
 'group_rank(ts_rank(assets, 252), industry)',
 'group_rank(ts_rank(assets, 252), subindustry)',
 'group_rank(ts_rank(assets, 252), sector)',
 'group_rank(ts_rank(assets, 504), market)',
 'group_rank(ts_rank(assets, 504), industry)',
 'group_rank(ts_rank(assets, 504), subindustry)',
 'group_rank(ts_rank(assets, 504), sector)',
 'group_rank(ts_rank(assets_curr, 252), market)',
 'group_rank(ts_rank(assets_curr, 252), industry)']

In [7]:
# 将 alpha 表达式与 settings 封装

alpha_list = [generate_sim_data_single(alpha_expression) for alpha_expression in alpha_expressions] 

alpha_simulate_id_list = []  # 模拟成功的 alpha id 列表

alpha_expression_simulate_list = [] # 模拟成功的 alpha 表达式列表

alpha_expression_simulate_fail_list = [] # 模拟但是报错了的 alpha 表达式列表

alpha_expression_simulate_error_list = [] # 因为其他原因没有模拟的 alpha 表达式列表

In [ ]:
# 把上面的内容单独写一个脚本并把输出写入到一个文档，文档命名为 alpha_template_id_padding.txt
# 下面的

In [8]:

def single_simulate(alpha):

    keep_alive = True
    try_cnt = 2

    while keep_alive and try_cnt > 0:
        try:
            simulation_response = s.post('https://api.worldquantbrain.com/simulations', json=alpha)
            simulation_progress_url = simulation_response.headers['Location']
            while True:
                simulation_progress = s.get(simulation_progress_url)
                if simulation_progress.headers.get("Retry-After", 0) == 0:
                    break
                sleep(float(simulation_progress.headers["Retry-After"]))
            
            response = simulation_progress.json()
            status = response.get("status", 0)
            if status != "COMPLETE":
                alpha_expression_simulate_fail_list.append(alpha['regular'])
                logging.warning(f"alpha expression: {alpha['regular']}, alpha id: {response['alpha']}, status: {response['status']}, message: {response['message']} \n {simulation_progress_url}")
            else:
                alpha_expression_simulate_list.append(alpha['regular'])
                alpha_simulate_id_list.append(response["alpha"])
            keep_alive = False
        except Exception as e:
            s = login()
            if try_cnt == 1:
                logging.error(f"{alpha['regular']} \n Error occurred: {e}")
                alpha_expression_simulate_error_list.append(alpha['regular'])
            try_cnt -= 1
    sleep(1)


In [ ]:
process_num = 3 
# alpha_list_sub = alpha_list[: 10] # 测试用，实际使用时可以注释掉
alpha_list_sub = [alpha for alpha in alpha_list if alpha['regular'] not in alpha_expression_simulate_list] # 过滤掉已经模拟过的 alpha 表达式
# 使用 ThreadPoolExecutor 来并行处理 alpha 表达式
with ThreadPoolExecutor(max_workers=process_num) as executor:
    futures = [executor.submit(single_simulate, alpha) for alpha in alpha_list_sub]

b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["

In [ ]:
alpha_simulate_id_list
alpha_expression_simulate_list
alpha_expression_simulate_fail_list
alpha_expression_simulate_error_list

[]